In [13]:
"""
VIX Article Relevance Ranker using Google Gemini Flash
Takes GNews query output and ranks articles by VIX prediction relevance.
"""

import os
import pandas as pd
import google.generativeai as genai
import json
from datetime import datetime
import time

# ============================================================================
# CONFIGURATION
# ============================================================================

# os.environ.get("GOOGLE_API_KEY")

# ============================================================================
# HELPER FUNCTIONS
# ============================================================================

def parse_published_date(published_str: str) -> str:
    """
    Convert published timestamp to date (YYYY-MM-DD).
    Handles format: "Tue, 01 Jul 2025 07:00:00 GMT"
    """
    try:
        # Parse the timestamp
        dt = datetime.strptime(published_str, "%a, %d %b %Y %H:%M:%S %Z")
        return dt.strftime('%Y-%m-%d')
    except Exception as e:
        print(f"Warning: Could not parse date '{published_str}': {e}")
        return None


def rank_articles_with_gemini(articles_df: pd.DataFrame, date: str) -> list:
    """
    Use Gemini to rank articles by VIX prediction relevance.
    Returns article indices in order from most to least relevant.
    
    Args:
        articles_df: DataFrame of articles for a single day
        date: The date (YYYY-MM-DD) for context
    
    Returns:
        List of article indices in descending order of relevance (most relevant first)
    """
    try:
        model = genai.GenerativeModel('gemini-2.0-flash-lite')
        genai.configure(api_key=GEMINI_API_KEY)
        
        # Format articles for prompt
        articles_text = ""
        for idx in articles_df.index:
            row = articles_df.loc[idx]
            articles_text += f"{idx}. {row['title']} (Source: {row.get('source', 'Unknown')})\n"
        
        prompt = f"""You are predicting next-day VIX (market volatility index) changes based on news from {date}.

RANK these articles from MOST to LEAST relevant for predicting tomorrow's VIX change.

MOST RELEVANT articles have:
- Surprise events (emergency Fed meetings, unexpected bank failures, sudden geopolitical escalations)
- New uncertainties (policy confusion, conflicting economic signals, unprecedented situations)
- Fear-inducing developments (systemic risks, contagion, market stress indicators)
- Breaking escalations (worsening crises, expanding conflicts, widening credit spreads)

LEAST RELEVANT articles have:
- Historical analysis or retrospectives
- Expected/scheduled events everyone knew about
- General market commentary without new information
- Stable trends or business-as-usual news

Articles:
{articles_text}

Think carefully about which articles create the MOST UNCERTAINTY and FEAR for markets.

Return ONLY a JSON array of indices ranked from most to least relevant:
[5, 12, 3, 8, 1, 15, ...]

Include ALL article indices in your ranking. No explanations, just the array."""

        response = model.generate_content(prompt)
        
        # Extract JSON from response
        text = response.text.strip()
        
        # Handle markdown code blocks
        if '```json' in text:
            text = text.split('```json')[1].split('```')[0].strip()
        elif '```' in text:
            text = text.split('```')[1].split('```')[0].strip()
        
        # Parse JSON
        ranked_indices = json.loads(text)
        
        # Validate that all indices are present
        expected_indices = set(articles_df.index.tolist())
        returned_indices = set(ranked_indices)
        
        if expected_indices != returned_indices:
            missing = expected_indices - returned_indices
            extra = returned_indices - expected_indices
            if missing:
                print(f"  ⚠️  Missing indices: {missing}, appending to end")
                ranked_indices.extend(list(missing))
            if extra:
                print(f"  ⚠️  Extra indices: {extra}, removing")
                ranked_indices = [idx for idx in ranked_indices if idx in expected_indices]
        
        return ranked_indices
    
    except json.JSONDecodeError as e:
        print(f"  ⚠️  JSON parse error for {date}: {e}")
        print(f"  Response was: {text[:200]}...")
        return []
    
    except Exception as e:
        print(f"  ⚠️  Gemini error for {date}: {e}")
        return []


def assign_ranks(ranked_indices: list, articles_df: pd.DataFrame) -> pd.Series:
    """
    Assign rank numbers based on Gemini's ordering.
    Rank 1 = most relevant, rank N = least relevant.
    
    Args:
        ranked_indices: List of article indices in order of relevance (most to least)
        articles_df: DataFrame of articles for this day
    
    Returns:
        Series with relevance_vix values (1, 2, 3, ..., N) for each article
    """
    # Initialize all as unranked (will use max rank + 1 for any failures)
    max_rank = len(articles_df)
    relevance = pd.Series(max_rank, index=articles_df.index, name='relevance_vix')
    
    if len(ranked_indices) == 0:
        # If ranking failed, assign sequential ranks based on dataframe order
        print(f"  ⚠️  Using default ordering")
        return pd.Series(range(1, len(articles_df) + 1), index=articles_df.index, name='relevance_vix')
    
    # Assign ranks based on position in ranked list
    for rank, idx in enumerate(ranked_indices, start=1):
        relevance[idx] = rank
    
    return relevance


# ============================================================================
# MAIN FUNCTION
# ============================================================================

def filter_vix_relevance(df: pd.DataFrame, delay_between_days: float = 2.0) -> pd.DataFrame:
    """
    Rank articles by VIX prediction relevance using Google Gemini.
    
    Process:
    1. Convert published to date (published_dt)
    2. For each date, use Gemini to rank articles by relevance
    3. Assign ranks: 1 (most relevant) to N (least relevant)
    
    Args:
        df: DataFrame from GNews query with columns: title, published, source, etc.
        delay_between_days: Seconds to wait between API calls for different dates
    
    Returns:
        DataFrame with added columns: published_dt, relevance_vix (rank 1 to N per day)
    """
    print("\n" + "="*70)
    print("VIX RELEVANCE RANKING WITH GOOGLE GEMINI FLASH")
    print("="*70)
    print(f"Input: {len(df)} articles")
    
    # Create copy to avoid modifying original
    df_result = df.copy()
    
    # Step 1: Parse published_at to date
    print("\n📅 Parsing dates...")
    df_result['published_dt'] = df_result['published'].apply(parse_published_date)
    
    # Drop rows with invalid dates
    invalid_dates = df_result['published_dt'].isna().sum()
    if invalid_dates > 0:
        print(f"  ⚠️  Dropping {invalid_dates} articles with invalid dates")
        df_result = df_result[df_result['published_dt'].notna()].copy()
    
    # Get unique dates
    unique_dates = sorted(df_result['published_dt'].unique())
    print(f"  ✓ Found {len(unique_dates)} unique dates")
    
    # Step 2: Rank articles for each date
    print(f"\n🤖 Ranking articles with Gemini Flash...")
    print(f"{'─'*70}")
    
    all_relevance = []
    
    for i, date in enumerate(unique_dates, 1):
        # Get articles for this date
        date_mask = df_result['published_dt'] == date
        date_articles = df_result[date_mask]
        
        print(f"\n[{i}/{len(unique_dates)}] {date} - {len(date_articles)} articles")
        
        # Rank with Gemini
        ranked_indices = rank_articles_with_gemini(date_articles, date)
        
        if ranked_indices:
            print(f"  ✓ Ranked {len(ranked_indices)} articles")
            
            # Show top 5 most relevant
            top_n = min(5, len(ranked_indices))
            print(f"  📰 Top {top_n} most relevant:")
            for rank in range(top_n):
                idx = ranked_indices[rank]
                title = date_articles.loc[idx, 'title']
                print(f"    {rank+1}. {title[:70]}...")
            
            # Assign ranks
            relevance = assign_ranks(ranked_indices, date_articles)
            
            print(f"  → Ranks assigned: 1 (most relevant) to {len(date_articles)} (least relevant)")
        else:
            # If ranking failed, assign sequential ranks
            relevance = assign_ranks([], date_articles)
            print(f"  ⚠️  Ranking failed, assigned default sequential order")
        
        all_relevance.append(relevance)
        
        # Rate limiting (except for last date)
        if i < len(unique_dates):
            time.sleep(delay_between_days)
    
    # Step 3: Combine relevance rankings
    print(f"\n{'─'*70}")
    print("📊 Combining results...")
    
    relevance_series = pd.concat(all_relevance)
    df_result['relevance_vix'] = relevance_series
    
    # Summary statistics
    print(f"\n{'='*70}")
    print("SUMMARY")
    print(f"{'='*70}")
    print(f"Total articles: {len(df_result)}")
    print(f"Dates processed: {len(unique_dates)}")
    print(f"Average articles per day: {len(df_result) / len(unique_dates):.1f}")
    
    # Show rank distribution
    print(f"\nRank distribution (across all days):")
    rank_counts = df_result['relevance_vix'].value_counts().sort_index()
    rank_ranges = [(1, 5), (6, 10), (11, 20), (21, 100)]
    for start, end in rank_ranges:
        count = rank_counts[(rank_counts.index >= start) & (rank_counts.index <= end)].sum()
        if count > 0:
            print(f"  Ranks {start:2d}-{end:2d}: {count:4d} articles")
    
    print(f"\nColumns added:")
    print(f"  - published_dt: Date extracted from published timestamp")
    print(f"  - relevance_vix: Rank within each day (1=most relevant)")
    print(f"{'='*70}\n")
    
    return df_result

In [14]:
import pandas as pd
from datetime import datetime, timedelta 

df_list = []
START_DATE = datetime(2025, 1, 1)
for i in range(2):
    DATE_I = START_DATE + timedelta(days=i)
    df_list.append(pd.read_csv(f'data/vix_training/{DATE_I.strftime('%Y-%m-%d')}.csv'))

df = pd.concat(df_list)
df.reset_index(inplace=True, drop=True)
# df['summary'] = df['summary'].fillna(df['title'])
# df['summary_len'] = df['summary'].str.len()
# df['full_text'] = df['full_text'].fillna(df['title'])
df['title_len'] = df['title'].str.len()
df = df.loc[df.groupby('id')['title_len'].idxmax()]
# df['published_dt'] = pd.to_datetime(df['published'], format='mixed').dt.strftime('%Y-%m-%d')

In [15]:
df_ranked = filter_vix_relevance(df)


VIX RELEVANCE RANKING WITH GOOGLE GEMINI FLASH
Input: 60 articles

📅 Parsing dates...
  ✓ Found 3 unique dates

🤖 Ranking articles with Gemini Flash...
──────────────────────────────────────────────────────────────────────

[1/3] 2025-01-01 - 17 articles
  ⚠️  Gemini error for 2025-01-01: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_input_token_count, limit: 0, model: gemini-2.0-flash-lite
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.0-flash-lite
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.0-flash-lite
Please retry in 32.351894765s. [links 